In [1]:
import numpy as np
import pandas as pd

from textwrap import wrap

import statsmodels.api as sm

import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

import ast

import warnings
warnings.filterwarnings('ignore')

__Import Dataset__

In [3]:
mov = pd.read_csv('movies_budget_adjusted.csv')
mov

,id,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,crew,keywords,Year,Ticket Price,ROI,Tickets Sold,Revenue in 2023,Month,CPI,Budget in 2023
0,10003,False,"{'id': 86224, 'name': 'The Saint Collection', ...",68000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,tt0120053,en,The Saint,"Ivan Tretiak, Russian Mafia boss who wants to ...",...,"[{'credit_id': '52fe43039251416c75000187', 'de...","[{'id': 220, 'name': 'berlin'}, {'id': 2139, '...",1997,4.59,0.736225,2.572185e+07,2.687934e+08,4,160.5,1.278807e+08
1,10155,False,NaN,19000000,"[{'id': 53, 'name': 'Thriller'}]",NaN,tt0120399,en,U Turn,When Bobby's car breaks down in the desert whi...,...,"[{'credit_id': '52fe43379251416c75007dc3', 'de...","[{'id': 1927, 'name': 'car breakdown'}, {'id':...",1997,4.59,-0.648311,1.455795e+06,1.521306e+07,10,160.5,3.573136e+07
2,10336,False,NaN,40000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,tt0120177,en,Spawn,After being murdered by corrupt colleagues in ...,...,"[{'credit_id': '52fe435b9251416c7500d4d5', 'de...","[{'id': 2095, 'name': 'anti hero'}, {'id': 428...",1997,4.59,1.196001,1.913726e+07,1.999844e+08,7,160.5,7.522393e+07
3,10351,False,"{'id': 135489, 'name': 'Wishmaster Collection'...",5000000,"[{'id': 27, 'name': 'Horror'}]",NaN,tt0120524,en,Wishmaster,The Djinn having been released from his ancien...,...,"[{'credit_id': '52fe435e9251416c7500dbb5', 'de...","[{'id': 657, 'name': 'fire'}, {'id': 2250, 'na...",1997,4.59,2.143822,3.424642e+06,3.578751e+07,9,160.5,9.402991e+06
4,10371,False,NaN,35000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt0119142,en,For Richer or Poorer,"Brad Sexton and his wife, Caroline, are wealth...",...,"[{'credit_id': '52fe43629251416c7500e327', 'de...","[{'id': 242, 'name': 'new york'}, {'id': 904, ...",1997,4.59,-0.096822,6.886977e+06,7.196891e+07,12,160.5,6.582093e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4508,60216,False,NaN,855000,"[{'id': 53, 'name': 'Thriller'}, {'id': 10752,...",NaN,tt0035884,en,Five Graves to Cairo,"The British Army, retreating ahead of victorio...",...,"[{'credit_id': '56c3c988c3a36871fd00058b', 'de...","[{'id': 612, 'name': 'hotel'}, {'id': 1160, 'n...",1943,0.29,0.403509,4.137931e+06,4.324138e+07,5,17.3,1.491733e+07
4509,618,False,NaN,100000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",NaN,tt0004972,en,The Birth of a Nation,The Birth of A Nation is a silent film from 19...,...,"[{'credit_id': '59d42f0ec3a368453f009d78', 'de...","[{'id': 529, 'name': 'ku klux klan'}, {'id': 5...",1915,0.07,109.000000,1.571429e+08,1.642143e+09,2,10.1,2.988475e+06
4510,70368,False,NaN,17311,"[{'id': 18, 'name': 'Drama'}]",NaN,tt0005078,en,The Cheat,"A venal, spoiled stockbroker's wife impulsivel...",...,"[{'credit_id': '52fe47f4c3a368484e0e2655', 'de...",[],1915,0.07,6.935128,1.962357e+06,2.050663e+07,12,10.1,5.173350e+05
4511,65203,False,NaN,379000,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,tt0019729,en,The Broadway Melody,"Harriet and Queenie Mahoney, a vaudeville act,...",...,"[{'credit_id': '52fe4701c3a368484e0afdcb', 'de...","[{'id': 4344, 'name': 'musical'}, {'id': 10229...",1929,0.35,10.498681,1.245143e+07,1.301174e+08,2,17.1,6.689815e+06


## Data Cleaning

__Convert all text columns in the format of dictionaries into lists__

In [4]:
# Extract text names from columns

# turn columns into lists containing dictionaries
mov['collection_1'] = [ast.literal_eval(i) for i in mov['belongs_to_collection'].fillna("[]")]
mov['gen_1'] = [ast.literal_eval(i) for i in mov['genres']]
mov['comp_1'] = [ast.literal_eval(i) for i in mov['production_companies'].fillna("[]")]
mov['coun_1'] = [ast.literal_eval(i) for i in mov['production_countries'].fillna("[]")]
mov['lang_1'] = [ast.literal_eval(i) for i in mov['spoken_languages'].fillna("[]")]
mov['cast_1'] = [ast.literal_eval(i) for i in mov['cast']]
mov['crew_1'] = [ast.literal_eval(i) for i in mov['crew']]
mov['key_1'] = [ast.literal_eval(i) for i in mov['keywords']]

# extract desired text from columns, works for all columns except belongs_to_collections
# returns new column containing only desired text and separate list of unique categories in the column
def extract(col):
    new_col = []
    unique_cat = []
    for d_list in col:
        new_row = []
        if (len(d_list) > 0):
            for dic in d_list:
                name = dic['name']
                new_row.append(name)
                if (name not in unique_cat):
                    unique_cat.append(name)
        new_col.append(np.asarray(new_row, dtype = object))
    return unique_cat, new_col

# use extract function on columns
gen_cat, gen_col = extract(mov['gen_1'])
comp_cat, comp_col = extract(mov['comp_1'])
coun_cat, coun_col = extract(mov['coun_1'])
lang_cat, lang_col = extract(mov['lang_1'])
cast_cat, cast_col = extract(mov['cast_1'])
crew_cat, crew_col = extract(mov['crew_1'])
key_cat, key_col = extract(mov['key_1'])

# extract desired text from belongs_to_collections
collect_col = []
collect_cat = []
for dic in mov['collection_1']:
    if (type(dic) != list):
        name = dic['name']
        collect_col.append(name)
        if (name not in collect_cat):
            collect_cat.append(name)
    else:
        collect_col.append('NaN')
collect_cat.append('NaN')

# add new clean columns to dataframe
mov['collect_clean'] = collect_col
mov['genres_clean'] = gen_col
mov['companies_clean'] = comp_col
mov['countries_clean'] = coun_col
mov['languages_clean'] = lang_col
mov['cast_clean'] = cast_col
mov['crew_clean'] = crew_col
mov['keywords_clean'] = key_col

# keep only first 3 names in cast and crew columns
cast_short = []
crew_short = []
for i in range(len(mov)):
    cast_short.append(mov['cast_clean'].iloc[i][0 : 3])
    crew_short.append(mov['crew_clean'].iloc[i][0 : 3])
mov['cast_short'] = cast_short
mov['crew_short'] = crew_short

mov

,id,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,collect_clean,genres_clean,companies_clean,countries_clean,languages_clean,cast_clean,crew_clean,keywords_clean,cast_short,crew_short
0,10003,False,"{'id': 86224, 'name': 'The Saint Collection', ...",68000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,tt0120053,en,The Saint,"Ivan Tretiak, Russian Mafia boss who wants to ...",...,The Saint Collection,"[Thriller, Action, Romance, Science Fiction, A...","[Paramount Pictures, Mace Neufeld Productions,...",[United States of America],"[English, Pусский]","[Val Kilmer, Elisabeth Shue, Rade Serbedzija, ...","[Phillip Noyce, Jonathan Hensleigh, Wesley Str...","[berlin, russia, gas, master thief, the saint]","[Val Kilmer, Elisabeth Shue, Rade Serbedzija]","[Phillip Noyce, Jonathan Hensleigh, Wesley Str..."
1,10155,False,NaN,19000000,"[{'id': 53, 'name': 'Thriller'}]",NaN,tt0120399,en,U Turn,When Bobby's car breaks down in the desert whi...,...,NaN,[Thriller],"[Illusion Entertainment Group, Phoenix Picture...","[France, United States of America]",[English],"[Sean Penn, Nick Nolte, Jennifer Lopez, Joaqui...","[Robert Richardson, Victor Kempster, Oliver St...","[car breakdown, wilderness, kiss, gambling deb...","[Sean Penn, Nick Nolte, Jennifer Lopez]","[Robert Richardson, Victor Kempster, Oliver St..."
2,10336,False,NaN,40000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,tt0120177,en,Spawn,After being murdered by corrupt colleagues in ...,...,NaN,"[Action, Adventure, Fantasy, Horror, Science F...","[New Line Cinema, Pull Down Your Pants Picture...",[United States of America],[English],"[Michael Jai White, Martin Sheen, John Leguiza...","[Todd McFarlane, Alan B. McElroy, Clint Goldma...","[anti hero, secret agent, fistfight, based on ...","[Michael Jai White, Martin Sheen, John Leguizamo]","[Todd McFarlane, Alan B. McElroy, Clint Goldman]"
3,10351,False,"{'id': 135489, 'name': 'Wishmaster Collection'...",5000000,"[{'id': 27, 'name': 'Horror'}]",NaN,tt0120524,en,Wishmaster,The Djinn having been released from his ancien...,...,Wishmaster Collection,[Horror],"[Live Entertainment, Image Organization, Pierr...",[United States of America],[English],"[Andrew Divoff, Tammy Lauren, Robert Englund, ...","[Pierre David, Robert Kurtzman, Peter Atkins, ...","[fire, menace, wish, djinn, violence]","[Andrew Divoff, Tammy Lauren, Robert Englund]","[Pierre David, Robert Kurtzman, Peter Atkins]"
4,10371,False,NaN,35000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt0119142,en,For Richer or Poorer,"Brad Sexton and his wife, Caroline, are wealth...",...,NaN,[Comedy],"[Universal Pictures, Yorktown Productions, The...",[United States of America],[English],"[Tim Allen, Kirstie Alley, Wayne Knight, Larry...","[Bryan Spicer, Jana Howington, Buzz Feitshans ...","[new york, pennsylvania, cheating, tax consult...","[Tim Allen, Kirstie Alley, Wayne Knight]","[Bryan Spicer, Jana Howington, Buzz Feitshans IV]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4508,60216,False,NaN,855000,"[{'id': 53, 'name': 'Thriller'}, {'id': 10752,...",NaN,tt0035884,en,Five Graves to Cairo,"The British Army, retreating ahead of victorio...",...,NaN,"[Thriller, War]",[Paramount Pictures],[United States of America],"[English, Deutsch]","[Franchot Tone, Anne Baxter, Akim Tamiroff, Er...","[Herbert Coleman, Billy Wilder, Billy Wilder, ...","[hotel, egypt, world war ii, german, grave, br...","[Franchot Tone, Anne Baxter, Akim Tamiroff]","[Herbert Coleman, Billy Wilder, Billy Wilder]"
4509,618,False,NaN,100000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",NaN,tt0004972,en,The Birth of a Nation,The Birth of A Nation is a silent film from 19...,...,NaN,"[Drama, History, War]",[Epoch Film Co.],[United States of America],[No Language],"[Lillian Gish, Mae Marsh, Henry B. Walthall, M...","[Erich von Stroheim, Thomas F. Dixon Jr., Thom...","[ku klux klan, southern usa, patriot

__Filter out Outliers__

In [9]:
mov = mov[(mov["Budget in 2023"] >= 1000) & (mov["Revenue in 2023"] >= 1000)]
mov = mov[(mov["Revenue in 2023"] <= 5.294129e+08)]

__Keep Only Relevant Columns__

In [15]:
mov = mov[["id", "release_date", "runtime", "title", "overview", "vote_average", "vote_count", "Year", "Month", "Revenue in 2023", "Budget in 2023", "overview", "collect_clean", "genres_clean_list", "companies_clean_list", "countries_clean_list", "keywords_clean_list", "cast_clean_list", "crew_clean_list", "cast_short_list", "crew_short_list"]]

__Save cleaned dataset as new csv file__

In [12]:
# set text columns as list format
def change(x):
    return list(x)

mov['genres_clean_list'] = mov['genres_clean'].apply(change)
mov['companies_clean_list'] = mov['companies_clean'].apply(change)
mov['countries_clean_list'] = mov['countries_clean'].apply(change)
mov['keywords_clean_list'] = mov['keywords_clean'].apply(change)
mov['cast_clean_list'] = mov['cast_clean'].apply(change)
mov['crew_clean_list'] = mov['crew_clean'].apply(change)
mov['cast_short_list'] = mov['cast_short'].apply(change)
mov['crew_short_list'] = mov['crew_short'].apply(change)

In [ ]:
mov.to_csv("cleaned_dataset.csv", index=False)

## Feature Transformation

__Frequency Encoding on Production Companies, Cast, and Crew__

In [13]:
# turn strings in csv file into usable lists
cols = ['genres_clean_list', 'companies_clean_list', 'countries_clean_list', 'keywords_clean_list', 'cast_clean_list', 'crew_clean_list', 'cast_short_list', 'crew_short_list']
for i in range(len(mov)):
    for c in cols:
        mov[c].iloc[i] = ast.literal_eval(mov[c].iloc[i])

In [14]:
def convert_list(col, total):
    unique = mov[col].apply(pd.Series).stack().unique() # unique names
    freq = np.zeros(len(unique))

    for row in mov[col]: 
        for i in range(len(unique)):
            if (unique[i] in row):
                freq[i] += 1

    frac = (freq * 1.0) / total

    mov_frac = []
    for row in mov[col]: 
        sums = 0
        for name in row:
            index = np.where(unique == name)[0][0]
            sums += frac[index]
        mov_frac.append(sums / 3)
    
    return mov_frac, unique

total_cast = len(mov) * 3
total_crew = len(mov) * 3
total_companies = 0
for row in mov['companies_clean_list']:
    for name in row:
        total_companies += 1

mov['cast_short_freq'], unique_cast = convert_list('cast_short_list', total_cast)
mov['crew_short_freq'], unique_crew = convert_list('crew_short_list', total_crew)
mov['companies_freq'], unique_companies = convert_list('companies_clean_list', total_companies)
mov